<a href="https://colab.research.google.com/github/SabaEtezadRazavi/NLSFML/blob/master/4_Full_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import timeit
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN

%alias_magic t timeit

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from google.colab import drive
drive.mount('/content/drive')

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.
Mounted at /content/drive


In [11]:
# It can be used to reconstruct the model identically.
from tensorflow import keras
model_multi = keras.models.load_model("drive/MyDrive/LSSML_project/results/cnn100-model1-multi.h5")
model_binary = keras.models.load_model("drive/MyDrive/LSSML_project/results/cnn100-model1-binary.h5")

model_multi.load_weights
model_binary.load_weights

<bound method Model.load_weights of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f6f1bbacf28>>

In [13]:
#Informations about each of the models
multi = h5py.File('drive/MyDrive/LSSML_project/results/cm-scores-model1-multi.h5', 'r')
binary = h5py.File('drive/MyDrive/LSSML_project/results/cm-scores-model1-binary.h5', 'r')
print(multi.keys())
print(binary.keys())


<KeysViewHDF5 ['a_test', 'a_total', 'a_train', 'cm_test', 'cm_total', 'cm_train']>
<KeysViewHDF5 ['a_test', 'a_total', 'a_train', 'cm_test', 'cm_total', 'cm_train']>


# Checking the trained models

In [18]:
#Multi
print('Accuracies:')
print('Multi classification')
a_test_multi = np.array(multi.get('a_test'))
a_train_multi = np.array(multi.get('a_train'))
a_total_multi = np.array(multi.get('a_total'))
print('Accuracy on the test set:',a_test_multi)
print('Accuracy on the train set:',a_train_multi)
print('Accuracy on the complete set of data:',a_total_multi)

#Binary
print('Binary classification')
a_test_binary = np.array(binary.get('a_test'))
a_train_binary = np.array(binary.get('a_train'))
a_total_binary = np.array(binary.get('a_total'))
print('Accuracy on the test set:',a_test_binary)
print('Accuracy on the train set:',a_train_binary)
print('Accuracy on the complete set of data:',a_total_binary)


Accuracies:
Multi classification
Accuracy on the test set: 96.30293524535067
Accuracy on the train set: 97.22784340361171
Accuracy on the complete set of data: 97.04285866332057
Binary classification
Accuracy on the test set: 85.84166666666667
Accuracy on the train set: 88.02916666666667
Accuracy on the complete set of data: 87.59166666666667


# First: Checking the models 

In [23]:
real_image= h5py.File('drive/MyDrive/LSSML_project/data/50-boxsample.h5', 'r')
real_image.keys()


<KeysViewHDF5 ['X', 'Y']>

In [24]:
X_real = real_image.get('X')
Y_real = real_image.get('Y')

print(X_real)
print(Y_real)

<HDF5 dataset "X": shape (50, 16, 16, 16, 4), type "<f8">
<HDF5 dataset "Y": shape (50, 2), type "<f8">


In [32]:
Y_pred_real_multi = np.argmax( model_multi.predict(X_real) , axis=1 )
Y_pred_real_binary = np.argmax( model_binary.predict(X_real) , axis=1 )

data = {'real halo':np.array(Y_real)[:,0], 'multi predicted halo':Y_pred_real_multi, 'binary predicted halo':Y_pred_real_binary}
results = pd.DataFrame(data = data)
results

,real halo,multi predicted halo,binary predicted halo
0,1.0,1,0
1,36.0,36,1
2,30.0,7,1
3,0.0,1,0
4,10.0,10,1
5,12.0,12,1
6,38.0,38,0
7,26.0,26,1
8,31.0,31,1
9,5.0,5,1


In [38]:
artificial_image= h5py.File('drive/MyDrive/LSSML_project/data/uniform-m=2000.h5', 'r')
artificial_image.keys()
X_artificial = artificial_image.get('X')

print(X_artificial)


<HDF5 dataset "X": shape (20, 16, 16, 16, 4), type "<f8">


In [39]:
Y_pred_art_multi = np.argmax( model_multi.predict(X_artificial) , axis=1 )
Y_pred_art_binary = np.argmax( model_binary.predict(X_artificial) , axis=1 )

data_art = { 'multi predicted halo':Y_pred_art_multi, 'binary predicted halo':Y_pred_art_binary}
results_art = pd.DataFrame(data = data_art)
results_art

,multi predicted halo,binary predicted halo
0,2,0
1,2,0
2,2,0
3,2,0
4,2,0
5,2,0
6,2,0
7,2,0
8,2,0
9,2,0


In [46]:
from sklearn.metrics import accuracy_score
a = accuracy_score( np.array(Y_real)[:,0],Y_pred_real_multi)
print('Accuracy for multi classfication is:', a*100)
Y_binary = np.where(np.array(Y_real)[:,0],1,0)
print(Y_binary)
a = accuracy_score(Y_binary,Y_pred_real_binary)
print('Accuracy for binary classfication is:', a*100)

Accuracy for multi classfication is: 90.0
[1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0]
Accuracy for binary classfication is: 90.0


# Prallelizing the two models

In [61]:
def classifier(X):
  halo_bin = np.zeros(len(X))
  print(halo_bin)
  Y_in = np.argmax( model_binary.predict(X) , axis=1 )
  print(Y_in)
  one = np.array([1])
  zero = np.array([0])
  idx_out = np.isin(Y_in, zero)
  idx_in = np.isin(Y_in, one)
  print(idx_in)
  Y_pred = np.argmax( model_multi.predict(X[idx_in]) , axis=1 )
  halo_bin[idx_in] = Y_pred
  return halo_bin


In [64]:
Y_pred = classifier(np.array(X_real))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 0]
[False  True  True False  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True False]


In [65]:
results['parallel prediction'] = Y_pred
results

,real halo,multi predicted halo,binary predicted halo,parallel prediction
0,1.0,1,0,0.0
1,36.0,36,1,36.0
2,30.0,7,1,7.0
3,0.0,1,0,0.0
4,10.0,10,1,10.0
5,12.0,12,1,12.0
6,38.0,38,0,0.0
7,26.0,26,1,26.0
8,31.0,31,1,31.0
9,5.0,5,1,5.0


In [66]:
acc = accuracy_score( np.array(Y_real)[:,0],Y_pred)
print('Accuracy for multi classfication is:', acc*100)

Accuracy for multi classfication is: 84.0


# Finalization and saving the function for future uses

In [ ]:
def classifier(X):
  from tensorflow import keras
  model_multi = keras.models.load_model("drive/MyDrive/LSSML_project/results/cnn100-model1-multi.h5")
  model_binary = keras.models.load_model("drive/MyDrive/LSSML_project/results/cnn100-model1-binary.h5")
  halo_bin = np.zeros(len(X))
  print(halo_bin)
  Y_in = np.argmax( model_binary.predict(X) , axis=1 )
  print(Y_in)
  one = np.array([1])
  zero = np.array([0])
  idx_out = np.isin(Y_in, zero)
  idx_in = np.isin(Y_in, one)
  print(idx_in)
  Y_pred = np.argmax( model_multi.predict(X[idx_in]) , axis=1 )
  halo_bin[idx_in] = Y_pred
  return halo_bin
